In [65]:
# Rev Apr 2021
# By Zhijing Eu (zhijingeu@yahoo.com)
# This notebook demonstrates how to use a Simulated Annealing Search method to 
# solve the Wedding Seating Arrangement Problem

# Code below is based heaviliy on http://linanqiu.github.io/2018/03/05/Wedding-Seat-Optimization/
# BUT...with ADDITIONAL code to help
# 1.Provide the option to either generate 'dummy' data for testing or allow users to import guest lists/conditions from the earlier Excel tool 
# 2.Check the logical consistency of the seating constraint conditions
# 3.View the results in a more user friendly way.

In [66]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
import random

STEP1 : DEFINE NO OF GUESTS , TABLE SIZE AND SEATING CONSTRAINTS

In [67]:
# #Uncomment This Cell To Use 
# #OPTION 1 this cell to generate a generic listing from scratch 

# noOfGuests=12
# table_size = 4
# guest_list=list(range(1, noOfGuests+1))
# guest_list=["Guest-"+str(x) for x in guest_list]

# # negative is good (those two like each other), positive is bad (those two hate each other)
# # default is 50 , can also use larger nos based on relationship e.g. immediate family -100, cousins/close friends : -70 etc
# relationships_edges = {
#     ('Guest-1', 'Guest-2'): -50,
#     ('Guest-4', 'Guest-5'): -50,
#     ('Guest-6', 'Guest-11'): 50,
#     ('Guest-11', 'Guest-12'): 50,
# }

In [68]:
#OPTION 2 Use this cell to import from xls file
InputFileName="Example_Guests96pax_TableSize8pax.xls"

table_size = pd.read_excel(InputFileName, 'TableSize').columns[1]

GuestListRaw = pd.read_excel(InputFileName, 'GuestList')

guest_list=GuestListRaw["Guest"].values.tolist()


RelMatrixRaw=GuestListRaw.dropna(thresh=2)

RelMatrixRaw
relationships_edges={}
Together1=RelMatrixRaw[["Guest","Junto1"]].dropna(thresh=2)
Together2=RelMatrixRaw[["Guest","Junto2"]].dropna(thresh=2)
Together3=RelMatrixRaw[["Guest","Junto3"]].dropna(thresh=2)
Together1.columns=["Guest","Together"]
Together2.columns=["Guest","Together"]
Together3.columns=["Guest","Together"]
Together=pd.concat([Together1,Together2,Together3])
Apart1=RelMatrixRaw[["Guest","Separado-1"]].dropna(thresh=2)
Apart2=RelMatrixRaw[["Guest","Separado-2"]].dropna(thresh=2)
Apart3=RelMatrixRaw[["Guest","Separado-3"]].dropna(thresh=2)
Apart1.columns=["Guest","Apart"]
Apart2.columns=["Guest","Apart"]
Apart3.columns=["Guest","Apart"]
Apart=pd.concat([Apart1,Apart2,Apart3])
for element in list(zip(Together["Guest"], Together["Together"])):
        relationships_edges.update({element:-50})
for element in list(zip(Apart["Guest"], Apart["Apart"])):
        relationships_edges.update({element:50})
        
print("Imported Seating Arrangement Constraints List")
print(" ")
print("Default Value For Together = -50 , Default Value For Apart = +50")
print("Uncomment Following Cell To Modify/Add/Delete")
print("E.g Can Increase Strength Of Relationship by Using 1-100 e.g. -100 very close friends, +100 very bitter enemies")
relationships_edges

Imported Seating Arrangement Constraints List
 
Default Value For Together = -50 , Default Value For Apart = +50
Uncomment Following Cell To Modify/Add/Delete
E.g Can Increase Strength Of Relationship by Using 1-100 e.g. -100 very close friends, +100 very bitter enemies


{('Ms.Annikki Pietikäinen', 'Ms.Line Kristiansen'): -50,
 ('Ms.Johanna Wechsler', 'Mr.Kristof Kuntz'): -50,
 ('Mr.Bao Chin', 'Mrs.Bao Chin'): -50,
 ('Mrs.Chinyere Onwuka', 'Ms.Quinzia Schiavone'): -50,
 ('Ms.Quinzia Schiavone', 'Mr.Ross Morrison'): -50,
 ('Mr.Ross Morrison', 'Mr.Yasser Hakimi'): -50,
 ('Ms.Johanna Wechsler', 'Ms.Živana Breceljnik'): -50,
 ('Mr.Ross Morrison', "Mr.Leal D'Aoust"): -50,
 ('Mr.Bao Chin', 'Ms.Miranda Zaragamba'): 50,
 ('Mr.Shamsuddi Desheriyev', 'Ms.Orzsebet Lendvai'): 50,
 ('Ms.Orzsebet Lendvai', 'Mr.Shamsuddi Desheriyev'): 50,
 ('Ms.Miranda Zaragamba', 'Mr.Bao Chin'): 50,
 ('Mr.Yasser Hakimi', 'Dr.Hólmfríður Örlygsdóttir'): 50,
 ('Dr.Hólmfríður Örlygsdóttir', 'Mr.Yasser Hakimi'): 50}

In [69]:
# #OPTION2.1 - Modify relationships if required  
# #Uncomment to use

# #Add New Relationship
# new1 = {('Mr.Yasser Hakimi', 'Mrs.Signe Olsen'): -50}
# new2 = {('Mr.Ross Morrison', 'Mrs.Signe Olsen'): 50}
# new3 = { ('Mrs.Bao Chin', 'Mr.Bao Chin'): -50}
# new4 = {('Ms.Line Kristiansen','Mr.Alistair Wallace'):-50}
# new5 = {('Mr.Alistair Wallace','Ms.Annikki Pietikäinen'):-50}
# #Modify Existing Relationship
# modify1={('Mr.Bao Chin', 'Mrs.Bao Chin'): -75,}

# changes=[new1,new2,new3,new4,new5,modify1]
                         
# for change in changes:
#     relationships_edges.update(change)
    
# #Delete Relationship
# # relationships_edges.pop(('Mr.Marco Sackville-Baggins', 'Mrs.Asaka Iwasaki'))
# # relationships_edges.pop(('Mr.Girma Yohannes', 'Mrs.Ute Eisenhower'))

# print("Updated Seating Arrangement Constraints List")
# relationships_edges

In [70]:
# OPTION 3 Use this to import ONLY The Constraints Matrix which is input DIRECTLY with -100 to +100 values 
# References Example_ManualConstraintsMatrixInput.xls file where you would leave the 
# relationships as BLANK in the Wedding_ Seating_Arrangement_Solver and fill in Example_ManualConstraintsMatrixInput.xls 
# instead 

# InputFileName2="Example_ManualConstraintsMatrixInput.xls"
# ConstraintsListRaw = pd.read_excel(InputFileName2, 'ConstraintMatrix')
# ConstraintsListRaw= ConstraintsListRaw.dropna(thresh=2)
# ConstraintsListRaw= ConstraintsListRaw.fillna(0)
# ConstraintsListRaw = ConstraintsListRaw[(ConstraintsListRaw.T != 0 ).any()]
# ConstraintsListRaw = ConstraintsListRaw.loc[:, (ConstraintsListRaw != 0).any(axis=0)]
# relationships_edges={}
# for column in list(ConstraintsListRaw.columns[1:]):
#     X=ConstraintsListRaw[column].values.tolist()
#     for i in range(0,len(X)):
#         if X[i] != 0:
#             relationships_edges.update({(column,ConstraintsListRaw["Guest"].values.tolist()[i]):X[i]})
# relationships_edges

STEP2 : SET UP RELEVANT FUNCTIONS FOR SIM ANNEALING CALCS

In [71]:
def reshape_to_table_seats(x):
    return x.reshape(table_count, len(guest_list))

def cost(x):
    table_seats = reshape_to_table_seats(x)
    # Convert to match dimensions and use proper matrix multiplication
    table_costs = np.dot(np.dot(table_seats, relationships_mat), table_seats.T)
    table_cost = np.trace(table_costs)
    return table_cost

def take_step(x):
    # Use numpy array instead of matrix
    table_seats = reshape_to_table_seats(x.copy())
    # randomly swap two guests
    table_from, table_to = np.random.choice(table_count, 2, replace=False)
    
    table_from_guests = np.where(table_seats[table_from] == 1)[0]
    table_to_guests = np.where(table_seats[table_to] == 1)[0]
    
    table_from_guest = np.random.choice(table_from_guests)
    table_to_guest = np.random.choice(table_to_guests)
    
    table_seats[table_from, table_from_guest] = 0
    table_seats[table_from, table_to_guest] = 1
    table_seats[table_to, table_to_guest] = 0
    table_seats[table_to, table_from_guest] = 1
    return table_seats.flatten()  # Return flattened array

def prob_accept(cost_old, cost_new, temp):
    return 1 if cost_new < cost_old else np.exp((cost_old - cost_new) / temp)

def anneal(pos_current, temp=1.0, temp_min=0.00001, alpha=0.9, n_iter=100, audit=False):
    pos_current = np.array(pos_current)  # Ensure input is numpy array
    cost_old = cost(pos_current)
    
    audit_trail = []
    
    while temp > temp_min:
        for i in range(n_iter):
            pos_new = take_step(pos_current)
            cost_new = cost(pos_new)
            p_accept = prob_accept(cost_old, cost_new, temp)
            if p_accept > np.random.random():
                pos_current = pos_new
                cost_old = cost_new
            if audit:
                audit_trail.append((cost_new, cost_old, temp, p_accept))
        temp *= alpha
    
    return pos_current, cost_old, audit_trail

STEP3 : CHECK INPUT (E.G AVOID UNEVEN NO OF SEATS -I.E FILL IN SPARE SEATS) AND LOGICALLY CONSISTENT SEATING CONSTRAINTS

In [72]:
#ADJUSTS THE SEAT COUNTS TO ENSURE ALL TABLES ARE FILLED

table_count = len(guest_list) // table_size
print("No Of Guests: "+str(len(guest_list)))
if len(guest_list)%table_size==0:
    blank_seats=[]
if len(guest_list)%table_size!=0:
    blank_seats=list(range(1,((table_count+1)*table_size-len(guest_list))+1))
    blank_seats=["Spare-"+str(x) for x in blank_seats]
guest_list=guest_list+blank_seats
len(guest_list)
table_count = len(guest_list) // table_size

print("No Of Seats Per Table: "+str(table_size))
print("Required No Of Tables: "+str(table_count))
print("No Of Spare Seats: "+str(len(blank_seats)))

No Of Guests: 96
No Of Seats Per Table: 8
Required No Of Tables: 12
No Of Spare Seats: 0


In [73]:
#Generate Relationship Matrix

temp_graph = nx.Graph()
for guest in guest_list:
    temp_graph.add_node(guest)
    
for k, v in relationships_edges.items():
    temp_graph.add_edge(k[0], k[1], weight=v)
    
relationships_mat_unnormed = nx.to_numpy_array(temp_graph.to_undirected(), nodelist=guest_list)
relationships_mat = relationships_mat_unnormed / 100

with np.printoptions(precision=1, suppress=True):
    print(relationships_mat)

[[ 0.  -0.5  0.  ...  0.   0.   0. ]
 [-0.5  0.   0.  ...  0.   0.   0. ]
 [ 0.   0.   0.  ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  0.   0.   0. ]
 [ 0.   0.   0.  ...  0.   0.   0. ]
 [ 0.   0.   0.  ...  0.   0.   0. ]]


In [74]:
#Lists relationships 
relationships_edges

{('Ms.Annikki Pietikäinen', 'Ms.Line Kristiansen'): -50,
 ('Ms.Johanna Wechsler', 'Mr.Kristof Kuntz'): -50,
 ('Mr.Bao Chin', 'Mrs.Bao Chin'): -50,
 ('Mrs.Chinyere Onwuka', 'Ms.Quinzia Schiavone'): -50,
 ('Ms.Quinzia Schiavone', 'Mr.Ross Morrison'): -50,
 ('Mr.Ross Morrison', 'Mr.Yasser Hakimi'): -50,
 ('Ms.Johanna Wechsler', 'Ms.Živana Breceljnik'): -50,
 ('Mr.Ross Morrison', "Mr.Leal D'Aoust"): -50,
 ('Mr.Bao Chin', 'Ms.Miranda Zaragamba'): 50,
 ('Mr.Shamsuddi Desheriyev', 'Ms.Orzsebet Lendvai'): 50,
 ('Ms.Orzsebet Lendvai', 'Mr.Shamsuddi Desheriyev'): 50,
 ('Ms.Miranda Zaragamba', 'Mr.Bao Chin'): 50,
 ('Mr.Yasser Hakimi', 'Dr.Hólmfríður Örlygsdóttir'): 50,
 ('Dr.Hólmfríður Örlygsdóttir', 'Mr.Yasser Hakimi'): 50}

In [75]:
#CHECKS FOR INCONSISTENCIES BETWEEN RELATIONSHIPS 

pairs=[]
rel_values=[]
for pair, rel_value in relationships_edges.items():
    pairs.append(pair)
    rel_values.append(rel_value)

pairs

print("Checking For Pair Inconsistencies")
print(" ")

sorted(pairs[0])==sorted(pairs[0])
rel_values[0]!=rel_values[0]

indicesWithIssues=[]

for i in range(0,len(pairs)):
    for j in range(0,len(pairs)):
        if sorted(pairs[i])==sorted(pairs[j]) and rel_values[i]!=rel_values[j]:
            if i not in indicesWithIssues and j not in indicesWithIssues:
                print("Inconsistency Detected Between:")
                print(pairs[i],":",rel_values[i])
                print(pairs[j],":",rel_values[j])
                print(" ")
                indicesWithIssues.append(i)
                indicesWithIssues.append(j)            
        else:
            pass

print(" ")
print("Checking For Triad Inconsistencies")
print(" ")

all_cliques= nx.enumerate_all_cliques(temp_graph)
triad_cliques=[x for x in all_cliques if len(x)==3 ]
checkSignForTriad=[]
for triad in triad_cliques:
    print("Identified A Triad Consisting Of :",triad)
    for i in range(0,len(pairs)):
        if sorted(triad[1:])==sorted(pairs[i]) or sorted(triad[:2])==sorted(pairs[i]) or sorted([triad[0],triad[2]])==sorted(pairs[i]):
            print(pairs[i],":",rel_values[i])
            checkSignForTriad.append(rel_values[i])
    if (checkSignForTriad[0]<0 and checkSignForTriad[1]<0 and checkSignForTriad[2]<0) or (checkSignForTriad[0]>=0 and checkSignForTriad[1]>=0 and checkSignForTriad[2]>=0):
        print("Triad OK!")
    else:
        print("Triad Inconsistent ! Please Re-Check As All Triangles Should Be ALL Together or ALL Apart")
        indicesWithIssues.append(i)
    checkSignForTriad=[]
    print(" ")

if indicesWithIssues!=[]:
    print("Warning ! If Not Corrected, Errors May Arise")
else:
    print("OK! No Inconsistencies Found")


Checking For Pair Inconsistencies
 
 
Checking For Triad Inconsistencies
 
OK! No Inconsistencies Found


In [76]:
# View Relationship Matrix
RelationshipMatrix=pd.DataFrame(relationships_mat)
RelationshipMatrix.index=guest_list
RelationshipMatrix.columns=guest_list
RelationshipMatrix = RelationshipMatrix[(RelationshipMatrix.T != 0).any()]
RelationshipMatrix = RelationshipMatrix.loc[:, (RelationshipMatrix != 0).any(axis=0)]
RelationshipMatrix

,Ms.Annikki Pietikäinen,Ms.Line Kristiansen,Ms.Johanna Wechsler,Mr.Kristof Kuntz,Ms.Živana Breceljnik,Mr.Bao Chin,Mr.Shamsuddi Desheriyev,Ms.Orzsebet Lendvai,Ms.Miranda Zaragamba,Mrs.Chinyere Onwuka,Ms.Quinzia Schiavone,Mr.Ross Morrison,Mr.Yasser Hakimi,Mr.Leal D'Aoust,Dr.Hólmfríður Örlygsdóttir,Mrs.Bao Chin
Ms.Annikki Pietikäinen,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ms.Line Kristiansen,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ms.Johanna Wechsler,0.0,0.0,0.0,-0.5,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mr.Kristof Kuntz,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ms.Živana Breceljnik,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mr.Bao Chin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,-0.5
Mr.Shamsuddi Desheriyev,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ms.Orzsebet Lendvai,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ms.Miranda Zaragamba,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mrs.Chinyere Onwuka,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0


#### STEP4 : RUN SIMULATION

In [77]:
#Generate An Initial (Seed) Random Seating Arrangement
s = list(range(table_count*table_size))
random.shuffle(s)
s = [ x+1 for x in s]

Table_Arrangement=pd.DataFrame(zip(guest_list,s),columns=["Guest Name","Assigned Seat No"])
Table_Arrangement["Assigned Table No"]=((Table_Arrangement["Assigned Seat No"]-1)//table_size)+1

# Table_Arrangement.sort_values(by=['Assigned Table No'])

for i in range(1,table_count+1):
    Table_Arrangement["Table No "+str(i)]=np.where(Table_Arrangement['Assigned Table No']!= i, 0, 1)
Table_Arrangement_Transpose=Table_Arrangement.T
Table_Arrangement_Transpose=Table_Arrangement_Transpose.tail(len(Table_Arrangement_Transpose)-3)
initial_random_arrangement=Table_Arrangement_Transpose.values
Table_Arrangement[["Guest Name","Assigned Table No"]]

,Guest Name,Assigned Table No
0,Ms.Annikki Pietikäinen,5
1,Ms.Line Kristiansen,7
2,Mr.Koppány Baranyi,11
3,Ms.Yukiko Koyama,6
4,Ms.Johanna Wechsler,12
...,...,...
91,Mrs.Bao Chin,1
92,Spare Seat 1,4
93,Spare Seat 2,8
94,Spare Seat 3,8


In [78]:
initial_random_arrangement_costs = np.matrix(initial_random_arrangement) * relationships_mat * initial_random_arrangement.T
print(initial_random_arrangement_costs)

print(np.trace(initial_random_arrangement_costs))

[[0.0 0.0 0.0 0.5 -0.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 -0.5 -0.5 0.0 0.0 0.0 -0.5]
 [0.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.5]
 [-0.5 0.0 0.0 0.0 1.0 0.0 -0.5 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.5 0.0 0.0 0.0 0.0]
 [0.0 0.0 -0.5 0.0 -0.5 0.0 0.0 0.0 0.5 0.0 0.0 -0.5]
 [0.0 0.0 -0.5 0.0 0.0 -0.5 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 -0.5 -0.5 0.0 0.0 -0.5 0.0 0.0 0.0 0.0 0.0]]
1.0


In [79]:
#NOTE - MAY TAKE TIME TO RUN IF NUMBERS ARE LARGE!

result = anneal(initial_random_arrangement,temp=1.0, temp_min=0.00001, alpha=0.9, n_iter=100, audit=False)

##### STEP5: VIEW RESULTS & PROPOSED SEATING ARRANGEMENT

In [80]:
print("Cost Function Of Optimized Seating Arrangement:",cost(result[0]),"vs. Initial Seed Cost Function Value Of",np.trace(initial_random_arrangement_costs))
print("NB: Lower Number = Better")

Cost Function Of Optimized Seating Arrangement: -8.0 vs. Initial Seed Cost Function Value Of 1.0
NB: Lower Number = Better


In [81]:
# print(cost(result[0]))
# print(result[0])
# audit_df = pd.DataFrame(result[2], columns=['cost_new', 'cost_old', 'temp', 'p_accept'])
# audit_df[['cost_old']].plot()
# audit_df[['temp']].plot()
# audit_df[['p_accept']].plot()

In [82]:
multiplier_table=[]
for i in range(1,table_count+1):
    multiplier_table.append([i])
    
suggested_arrangement=pd.DataFrame(np.array(result[0])*np.array(multiplier_table)).T
suggested_arrangement.columns=Table_Arrangement.columns[-table_count:]
suggested_arrangement["Assigned Table No"]=suggested_arrangement.sum(axis=1)
suggested_arrangement['Guest Name']=Table_Arrangement['Guest Name']
suggested_arrangement=suggested_arrangement[["Guest Name","Assigned Table No"]]
display(suggested_arrangement)
display("Ordered By Table No")
suggested_arrangement_by_tableNo=suggested_arrangement.sort_values(by=['Assigned Table No'])
suggested_arrangement_by_tableNo[["Assigned Table No","Guest Name"]]
display(suggested_arrangement_by_tableNo)

,Guest Name,Assigned Table No
0,Ms.Annikki Pietikäinen,0
1,Ms.Line Kristiansen,0
2,Mr.Koppány Baranyi,0
3,Ms.Yukiko Koyama,0
4,Ms.Johanna Wechsler,0
...,...,...
1147,NaN,0
1148,NaN,0
1149,NaN,78
1150,NaN,0


'Ordered By Table No'

,Guest Name,Assigned Table No
0,Ms.Annikki Pietikäinen,0
740,NaN,0
741,NaN,0
743,NaN,0
744,NaN,0
...,...,...
425,NaN,78
684,NaN,78
138,NaN,78
708,NaN,78


In [83]:
suggested_arrangement

,Guest Name,Assigned Table No
0,Ms.Annikki Pietikäinen,0
1,Ms.Line Kristiansen,0
2,Mr.Koppány Baranyi,0
3,Ms.Yukiko Koyama,0
4,Ms.Johanna Wechsler,0
...,...,...
1147,NaN,0
1148,NaN,0
1149,NaN,78
1150,NaN,0


In [84]:
#Filtered To Guests With Seating Conditions
GuestsWSeatingConditions=list(zip(*list(relationships_edges.keys())))[0]+list(zip(*list(relationships_edges.keys())))[1]
GuestsWSeatingConditions = list(dict.fromkeys(GuestsWSeatingConditions))
suggested_arrangement[suggested_arrangement['Guest Name'].isin(GuestsWSeatingConditions)]

,Guest Name,Assigned Table No
0,Ms.Annikki Pietikäinen,0
1,Ms.Line Kristiansen,0
4,Ms.Johanna Wechsler,0
5,Mr.Kristof Kuntz,0
6,Ms.Živana Breceljnik,0
8,Mr.Bao Chin,0
22,Mr.Shamsuddi Desheriyev,0
23,Ms.Orzsebet Lendvai,0
25,Ms.Miranda Zaragamba,0
30,Mrs.Chinyere Onwuka,0


In [85]:
FinalSeatingArrangementDF=suggested_arrangement
FinalSeatingArrangementDF.columns=["Guest","Assigned Table No"]

In [87]:
FinalSeatingArrangementWConditions = pd.merge(FinalSeatingArrangementDF,RelMatrixRaw,on ='Guest',how='left')

FinalSeatingArrangementWConditions =FinalSeatingArrangementWConditions[["Guest","Assigned Table No","Junto1","Junto2","Junto3","Separado-1","Separado-2","Separado-3"]]

FinalSeatingArrangementWConditions.sort_values(by=['Assigned Table No'])

,Guest,Assigned Table No,Junto1,Junto2,Junto3,Separado-1,Separado-2,Separado-3
0,Ms.Annikki Pietikäinen,0,Ms.Line Kristiansen,NaN,NaN,NaN,NaN,NaN
740,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
741,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
743,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
744,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
425,NaN,78,NaN,NaN,NaN,NaN,NaN,NaN
684,NaN,78,NaN,NaN,NaN,NaN,NaN,NaN
138,NaN,78,NaN,NaN,NaN,NaN,NaN,NaN
708,NaN,78,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
#EXPORT RESULTS TO CSV
FinalSeatingArrangementWConditions.to_csv("Output_SimAnn_Final"+InputFileName.strip('.xls')+".csv")

In [95]:
FinalSeatingArrangementWConditions.sort_values(by=['Assigned Table No']).dropna(thresh=3)

,Guest,Assigned Table No,Junto1,Junto2,Junto3,Separado-1,Separado-2,Separado-3
0,Ms.Annikki Pietikäinen,0,Ms.Line Kristiansen,NaN,NaN,NaN,NaN,NaN
4,Ms.Johanna Wechsler,0,Mr.Kristof Kuntz,Ms.Živana Breceljnik,NaN,NaN,NaN,NaN
8,Mr.Bao Chin,0,Mrs.Bao Chin,NaN,NaN,Ms.Miranda Zaragamba,NaN,NaN
33,Mr.Yasser Hakimi,0,NaN,NaN,NaN,Dr.Hólmfríður Örlygsdóttir,NaN,NaN
32,Mr.Ross Morrison,0,Mr.Yasser Hakimi,Mr.Leal D'Aoust,NaN,NaN,NaN,NaN
31,Ms.Quinzia Schiavone,0,Mr.Ross Morrison,NaN,NaN,NaN,NaN,NaN
30,Mrs.Chinyere Onwuka,0,Ms.Quinzia Schiavone,NaN,NaN,NaN,NaN,NaN
25,Ms.Miranda Zaragamba,0,NaN,NaN,NaN,Mr.Bao Chin,NaN,NaN
23,Ms.Orzsebet Lendvai,0,NaN,NaN,NaN,Mr.Shamsuddi Desheriyev,NaN,NaN
22,Mr.Shamsuddi Desheriyev,0,NaN,NaN,NaN,Ms.Orzsebet Lendvai,NaN,NaN
